<a href="https://colab.research.google.com/github/JebaMercy14/OpenAI-Pretrained-Model/blob/main/stable_diffusion_with_chatgpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation

In [10]:
!pip install -q gradio
!pip install -q pyChatGPT
!pip install -q git+https://github.com/openai/whisper.git
!pip install -q --upgrade git+https://github.com/huggingface/diffusers.git transformers accelerate scipy

  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


# Imports

In [11]:
import whisper
import gradio as gr 
import time
import warnings
import torch
from pyChatGPT import ChatGPT
from diffusers import StableDiffusionPipeline, EulerDiscreteScheduler

# Defining Variables

In [ ]:
warnings.filterwarnings("ignore")

In [ ]:
secret_token = "" #insert your secret_token


In [ ]:
model = whisper.load_model("base")

100%|████████████████████████████████████████| 139M/139M [00:01<00:00, 125MiB/s]


In [ ]:
model.device

device(type='cuda', index=0)

In [ ]:
model_id = "stabilityai/stable-diffusion-2"
scheduler = EulerDiscreteScheduler.from_pretrained(model_id, 
                                                   subfolder="scheduler")

pipe = StableDiffusionPipeline.from_pretrained(model_id, 
                                               scheduler=scheduler, 
                                               revision="fp16", 
                                               torch_dtype=torch.float16)
pipe = pipe.to("cuda")

Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

# Transcribe Function

In [ ]:
def transcribe(audio):

    # load audio and pad/trim it to fit 30 seconds
    audio = whisper.load_audio(audio)
    audio = whisper.pad_or_trim(audio)

    # make log-Mel spectrogram and move to the same device as the model
    mel = whisper.log_mel_spectrogram(audio).to(model.device)

    # detect the spoken language
    _, probs = model.detect_language(mel)

    # decode the audio
    options = whisper.DecodingOptions()
    result = whisper.decode(model, mel, options)
    result_text = result.text

    # Pass the generated text to Audio
    chatgpt_api = ChatGPT(secret_token)
    resp = chatgpt_api.send_message(result_text)
    out_result = resp['message']

    out_image = pipe(out_result, height=768, width=768).images[0]

    return [result_text, out_result, out_image]

# Gradio Interface

In [13]:
output_1 = gr.Textbox(label="Speech to Text")
output_2 = gr.Textbox(label="ChatGPT Output")
output_3 = gr.Image(label="Diffusion Output")

gr.Interface(
    title = 'OpenAI Whisper and ChatGPT ASR Gradio Web UI', 
    fn=transcribe, 
    inputs=[
        gr.inputs.Audio(source="microphone", type="filepath")
    ],

    outputs=[
        output_1,  output_2, output_3
    ],
    launch=True).launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>